# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
city_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_df.head ()


,City,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,Country,Date
0,Kapaa,22.0752,-159.3190,84.11,70,89,5.99,US,1665435244
1,São João da Barra,-21.6403,-41.0511,80.69,84,9,7.40,BR,1665435244
2,Dingle,10.9995,122.6711,74.57,91,50,1.10,PH,1665435245
3,Punta Arenas,-53.1500,-70.9167,53.71,43,40,32.21,CL,1665435200
4,Constantine,36.3650,6.6147,63.81,82,40,9.22,DZ,1665435081


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key = g_key)

locations = city_df[["Latitude", "Longitude"]]
weight = city_df["Humidity"]
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=4)
heatmap_layer = gmaps.heatmap_layer(locations, weights = weight)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
city_df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             550 non-null    object 
 1   Latitude         550 non-null    float64
 2   Longitude        550 non-null    float64
 3   Max Temperature  550 non-null    float64
 4   Humidity         550 non-null    int64  
 5   Cloud Coverage   550 non-null    int64  
 6   Wind Speed       550 non-null    float64
 7   Country          545 non-null    object 
 8   Date             550 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 38.8+ KB


In [10]:
# no null values 

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions:
Temp 72-85
Wind 3-7 knots 
Cloud 0-3 oktas (0-30) 
I like dry heat so between 15% and 35% humidity.

In [11]:
temp = (city_df["Max Temperature"] >= 72)&(city_df["Max Temperature"] <= 85)
wind = (city_df["Wind Speed"] >= 3)&(city_df["Wind Speed"] <= 7)
cloud = (city_df["Cloud Coverage"] >= 0) &(city_df["Cloud Coverage"] <= 30)
humidity = (city_df["Humidity"] >= 15) & (city_df["Humidity"] <= 35)
weather = city_df.loc[temp & wind & cloud & humidity]
 
weather



,City,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,Country,Date
323,Nizwá,22.9333,57.5333,75.92,22,14,3.13,OM,1665435282
525,Lexington,37.9887,-84.4777,74.14,30,0,3.00,US,1665435308
539,Santa Fe,-31.6333,-60.7000,72.01,24,2,5.99,AR,1665435097


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = weather

# add "Hotel Name" column
hotel_df["hotel name"] = " "
hotel_df.head ()

C:\Users\Helen\AppData\Local\Temp\ipykernel_20152\2649251130.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["hotel name"] = " "


,City,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,Country,Date,hotel name
323,Nizwá,22.9333,57.5333,75.92,22,14,3.13,OM,1665435282,
525,Lexington,37.9887,-84.4777,74.14,30,0,3.00,US,1665435308,
539,Santa Fe,-31.6333,-60.7000,72.01,24,2,5.99,AR,1665435097,


In [14]:
hotel_df=hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})

In [16]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    coords = (f"{lat},{lng}")

    parameters = {
    "key": g_key,
    # Set parameters to search for hotels with 5000 meters 
        # the assingment page says 10,000 metres
    "radius": 5000,
    "location": coords,
    "type": "lodging"
   }

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response_json = requests.get(url, params = parameters).json()

    print(f'Getting your results {index}: {city}')

    try:
        hotel_name = hotel_df.loc[index,"hotel name"] = response_json["results"][0]["name"]
        print(f"The closest hotel to {city} is the: {hotel_name}")
        
    except (KeyError, IndexError):
        print("No hotels available.")
    print("------------")

hotel_df.head ()

Getting your results 323: Nizwá
The closest hotel to Nizwá is the: Orient Apartments
------------
Getting your results 525: Lexington
The closest hotel to Lexington is the: DoubleTree Suites by Hilton Hotel Lexington
------------
Getting your results 539: Santa Fe
The closest hotel to Santa Fe is the: Conquistador Hotel
------------


,City,Lat,Lng,Max Temperature,Humidity,Cloud Coverage,Wind Speed,Country,Date,hotel name
323,Nizwá,22.9333,57.5333,75.92,22,14,3.13,OM,1665435282,Orient Apartments
525,Lexington,37.9887,-84.4777,74.14,30,0,3.00,US,1665435308,DoubleTree Suites by Hilton Hotel Lexington
539,Santa Fe,-31.6333,-60.7000,72.01,24,2,5.99,AR,1665435097,Conquistador Hotel


In [13]:
#hotel_df=hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
columns = ["Lat", "Lng"]
locations = hotel_df[columns].astype(float)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))